# Home Work

In [1]:
# 行列計算のため
import numpy as np
#csv拡張子を読む、データフレーム形式になる
import pandas as pd
#図を描くため
import matplotlib.pyplot as plt
#統計モデル
import statsmodels.api as sm
import statsmodels.formula.api as smf
#警告メッセージ非表示
import warnings
warnings.filterwarnings('ignore')
from scipy.stats import logistic
import scipy.stats

### csvファイル読む

In [2]:
data = pd.read_csv("GujaratiTable8_1.csv")
data.head()

,educ,age,income,pcigs79,ageeduc,smoker,educincome
0,12.0,21,8500,60.6,252.0,0,102000
1,15.0,28,12500,60.6,420.0,1,187500
2,10.0,67,12500,60.6,670.0,0,125000
3,12.0,20,12500,60.6,240.0,0,150000
4,12.0,32,20000,60.6,384.0,1,240000


In [3]:
#表を作り、def関数を作り、（df＿fa,info）自分設定した引数、
def feature_summary(df_fa, info):
    print('DataFrame shape')
    print('rows: ', df_fa.shape[0])
    print('cols: ', df_fa.shape[1])
#列の名称
    col_list = ['info', 'Null', 'Unique_N', 'Type', 'Mean', 'Median', 'Std', 'Skewness', 'Sample_values']
#dataframeを作成
    df = pd.DataFrame(index=df_fa.columns, columns=col_list)   
    df['Null'] = list([len(df_fa[col][df_fa[col].isnull()]) for i, col in enumerate(df_fa.columns)])
    df['Unique_N'] = list([len(df_fa[col].unique()) for i, col in enumerate(df_fa.columns)])
    df['Type'] = list([df_fa[col].dtype for i, col in enumerate(df_fa.columns)])
    df['info'] = info
    for i, col in enumerate(df_fa.columns):
        if 'float' in str(df_fa[col].dtype) or 'int' in str(df_fa[col].dtype):
            df.at[col, 'Mean'] = df_fa[col].mean()
            df.at[col, 'Median'] = df_fa[col].median()
            df.at[col, 'Std'] = df_fa[col].std()
            df.at[col, 'Skewness'] = df_fa[col].skew()
        df.at[col, 'Sample_values'] = list(df_fa[col].unique())
    return df.fillna('-')

In [4]:
feature_summary(data, ['教育年数', '年齢', '世代所得', '79年タバコ価額', '年齢×教育', '喫煙者(0,1)', '教育年数×世代所得'])

DataFrame shape
rows:  1196
cols:  7


,info,Null,Unique_N,Type,Mean,Median,Std,Skewness,Sample_values
educ,教育年数,0,10,float64,12.221154,12.0,3.275847,-0.539669,"[12.0, 15.0, 10.0, 13.5, 0.0, 16.0, 2.5, 8.0, ..."
age,年齢,0,71,int64,41.806856,39.0,17.056941,0.407220,"[21, 28, 67, 20, 32, 46, 58, 27, 83, 24, 43, 1..."
income,世代所得,0,11,int64,19304.765886,20000.0,9083.511331,-0.201524,"[8500, 12500, 20000, 30000, 6500, 3500, 4500, ..."
pcigs79,79年タバコ価額,0,42,float64,60.984950,62.1,4.848666,-1.009428,"[60.6, 60.9, 60.8, 61.5, 62.1, 54.6, 67.0, 62...."
ageeduc,年齢×教育,0,257,float64,498.954849,472.5,227.468150,0.556947,"[252.0, 420.0, 670.0, 240.0, 384.0, 690.0, 696..."
smoker,"喫煙者(0,1)",0,2,int64,0.380435,0.0,0.485697,0.493170,"[0, 1]"
educincome,教育年数×世代所得,0,75,int64,246773.620401,240000.0,145644.926730,0.301007,"[102000, 187500, 125000, 150000, 240000, 45000..."


## ロジスティック回帰モデル

### 出力結果 

In [5]:
fit = smf.logit('smoker ~ educ + age + income + pcigs79 + educ:income', data=data).fit()
#fit = smf.logit('smoker ~ educ + age + income + pcigs79 + educincome', data=data).fit()
print(fit.summary2())

Optimization terminated successfully.
         Current function value: 0.639208
         Iterations 5
                         Results: Logit
Model:              Logit            No. Iterations:   5.0000   
Dependent Variable: smoker           Pseudo R-squared: 0.038    
Date:               2018-10-10 23:02 AIC:              1540.9851
No. Observations:   1196             BIC:              1571.5056
Df Model:           5                Log-Likelihood:   -764.49  
Df Residuals:       1190             LL-Null:          -794.47  
Converged:          1.0000           Scale:            1.0000   
-----------------------------------------------------------------
              Coef.   Std.Err.     z     P>|z|    [0.025   0.975]
-----------------------------------------------------------------
Intercept     1.0932    0.9557   1.1439  0.2527  -0.7799   2.9663
educ          0.0395    0.0425   0.9281  0.3533  -0.0439   0.1228
age          -0.0183    0.0038  -4.8113  0.0000  -0.0257  -0.0108
income 

限界効果：

\begin{align*}
\frac{d Pr(Y_i = 1)}{d X} = \beta f(-\alpha - \beta X)
\end{align*}

In [6]:
a = fit.params[0]
ß = fit.params[1:].values
X = data.loc[:, ['educ', 'age', 'income', 'pcigs79', 'educincome']].values
print("ß shape: ", ß.shape)
print("X shape: ", X.shape)

ß shape:  (5,)
X shape:  (1196, 5)


In [7]:
print(a)
print(X.dot(ß))
f_inner = -(a + X.dot(ß))

1.0931856672926148
[-1.17786168 -1.44430757 -1.88789521 ... -1.44494684 -1.82790169
 -1.40050539]


In [8]:
from scipy.stats import logistic
lg_pdf = logistic.pdf(f_inner)
lg_pdf = lg_pdf.reshape(1196,1)
me = ß.reshape(5,1).dot(lg_pdf.T)

In [9]:
variables_nm = ['educ', 'age', 'income', 'pcigs79', 'educincome']
me_table = pd.DataFrame(me.T, columns=variables_nm)
me_table.head()

,educ,age,income,pcigs79,educincome
0,0.009846,-0.004555,0.000024,-0.005417,-0.000002
1,0.009566,-0.004426,0.000023,-0.005263,-0.000002
2,0.008457,-0.003913,0.000020,-0.004653,-0.000002
3,0.009859,-0.004561,0.000024,-0.005424,-0.000002
4,0.009744,-0.004508,0.000023,-0.005361,-0.000002


In [10]:
me_table.mean()

educ          0.008847
age          -0.004093
income        0.000021
pcigs79      -0.004867
educincome   -0.000002
dtype: float64

In [11]:
#検証
fit.get_margeff(at='overall').summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
        Logit Marginal Effects       
=====================================
Dep. Variable:                 smoker
Method:                          dydx
At:                           overall
===============================================================================
                 dy/dx    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------
educ            0.0088      0.010      0.929      0.353      -0.010       0.028
age            -0.0041      0.001     -4.993      0.000      -0.006      -0.002
income        2.13e-05   5.91e-06      3.602      0.000    9.71e-06    3.29e-05
pcigs79        -0.0049      0.003     -1.741      0.082      -0.010       0.001
educ:income  -1.67e-06    4.7e-07     -3.553      0.000   -2.59e-06   -7.49e-07
===============================================================================
"""

## プロビットモデル

In [12]:
fit_pb = smf.probit('smoker ~ educ + age + income + pcigs79 + educ:income', data=data).fit()
print(fit_pb.summary2())

Optimization terminated successfully.
         Current function value: 0.638942
         Iterations 5
                        Results: Probit
Model:              Probit           No. Iterations:   5.0000   
Dependent Variable: smoker           Pseudo R-squared: 0.038    
Date:               2018-10-10 23:02 AIC:              1540.3489
No. Observations:   1196             BIC:              1570.8693
Df Model:           5                Log-Likelihood:   -764.17  
Df Residuals:       1190             LL-Null:          -794.47  
Converged:          1.0000           Scale:            1.0000   
-----------------------------------------------------------------
              Coef.   Std.Err.     z     P>|z|    [0.025   0.975]
-----------------------------------------------------------------
Intercept     0.6821    0.5873   1.1613  0.2455  -0.4690   1.8331
educ          0.0242    0.0260   0.9322  0.3512  -0.0267   0.0751
age          -0.0114    0.0023  -4.8809  0.0000  -0.0160  -0.0068
income 

限界効果
\begin{align*}
\frac{d Pr(Y_i = 1)}{d X} = \tau f(-\gamma - \tau X)
\end{align*}

In [13]:
r = fit_pb.params[0]
t = fit_pb.params[1:].values
X = data.loc[:, ['educ', 'age', 'income', 'pcigs79', 'educincome']].values
print("t shape: ", t.shape)
print("X shape: ", X.shape)

t shape:  (5,)
X shape:  (1196, 5)


In [14]:
print(r)
print(X.dot(t))
f_inner = -(r + X.dot(t))

0.6820504936358941
[-0.73405038 -0.89820236 -1.17864672 ... -0.90028482 -1.14118038
 -0.87154293]


In [15]:
from scipy.stats import norm
pb_pdf = norm.pdf(f_inner)
pb_pdf = pb_pdf.reshape(1196,1)
me_pb = t.reshape(5,1).dot(pb_pdf.T)

In [16]:
variables_nm = ['educ', 'age', 'income', 'pcigs79', 'educincome']
me_table_pb = pd.DataFrame(me_pb.T, columns=variables_nm)
me_table_pb.head()

,educ,age,income,pcigs79,educincome
0,0.009642,-0.004535,0.000023,-0.005354,-0.000002
1,0.009432,-0.004436,0.000023,-0.005237,-0.000002
2,0.008535,-0.004014,0.000020,-0.004739,-0.000002
3,0.009651,-0.004539,0.000023,-0.005359,-0.000002
4,0.009563,-0.004498,0.000023,-0.005310,-0.000002


In [17]:
me_table_pb.mean()

educ          0.008834
age          -0.004155
income        0.000021
pcigs79      -0.004905
educincome   -0.000002
dtype: float64

In [18]:
#検証
fit_pb.get_margeff(at='overall').summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
       Probit Marginal Effects       
=====================================
Dep. Variable:                 smoker
Method:                          dydx
At:                           overall
===============================================================================
                 dy/dx    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------
educ            0.0088      0.009      0.933      0.351      -0.010       0.027
age            -0.0042      0.001     -5.037      0.000      -0.006      -0.003
income       2.118e-05   5.81e-06      3.646      0.000    9.79e-06    3.26e-05
pcigs79        -0.0049      0.003     -1.747      0.081      -0.010       0.001
educ:income -1.661e-06   4.61e-07     -3.607      0.000   -2.56e-06   -7.59e-07
===============================================================================
"""

## 解釈

- ロジスティック回帰モデル

<p>ロジスティック回帰モデルでは、限界効果から見ると、年齢と収入と教育年数×収入所得は5%有意な結果が得られた。年齢は喫煙者に負の影響を及ぼし、年齢は1歳が増えるにつれて、喫煙者の確率が0.4%減少する傾向が見られる。一方、収入については喫煙率に正の影響を及ぼし、教育年数と収入所得の交互作用は喫煙者に負の有意な結果が得られるが、数値は限りなく0に近いため、実質的有意性があるかどうか、検討する必要があると思われる。</p>
- プロビット回帰モデル

<p>プロビット回帰モデルでは、年齢と収入と教育年数×収入所得は5%有意な結果が得られた。ロジスティック回帰モデルもちほとんど同じである。</p>